In [4]:
import pathlib

External packages.

In [5]:
import ipywidgets
import itk
import numpy as np
from matplotlib import pyplot as plt

Utility functions.

In [6]:
def edgemap(image):
    from scipy.ndimage import sobel
    dx = sobel(image, axis=0)
    dy = sobel(image, axis=1)
    return np.hypot(dx, dy)

In [7]:
def imread(filepath):
    image = itk.imread(str(filepath))
    size = np.asarray(itk.size(image))
    spacing = np.asarray(itk.spacing(image))
    center = -(size - 1) * spacing / 2
    orientation = itk.array_from_matrix(image.GetDirection())
    image.SetOrigin(orientation @ center)
    image = itk.cast_image_filter(image, ttype=(type(image),itk.Image[itk.F,3]))
    return image

In [8]:
def mask(fg, bg=(0,0,0), alpha=1, n=256):
    from matplotlib import colors
    fg = colors.to_rgb(fg)
    bg = colors.to_rgb(bg)
    cmap = colors.LinearSegmentedColormap.from_list('binary', (bg,fg), n)
    cmap._init()
    cmap._lut[:,-1] = np.linspace(0, alpha, cmap.N + 3)
    return cmap

open CT file 

In [9]:
# File path. 
ct_filepath_cadaver = pathlib.Path(r"C:\Users\asong18\Desktop\Deep_RL_View_Planning\4_1mm.mha")


In [10]:
ct = imread(ct_filepath_cadaver)

# incorporate translation in order to align the images. 
# transform = itk.AffineTransform.D3.New()
# transform.SetTranslation((0, 13, 0))

# Changing the pixels (resolution and dimensions) of an US to match CT. This is the process of resampling. Interpolated using nearest neighbor?
# Also applies the transformation mentioned above. 
# ct = itk.resample_image_filter(ct, output_parameters_from_image=us, transform=transform)
# ct_bone = np.array(ct)
# ct_bone[ct_bone < 200] = 0
# index = ct.shape[0] // 2

In [11]:
index = ct.shape[0] // 2
fig, axs = plt.subplots(ncols=1, figsize=(12,3), subplot_kw={'xticks':(),'yticks':()})
plt.subplots_adjust(wspace=0.1)
plt.close(fig)
# this shows the Sagittal slice. 
im0 = axs.imshow(ct[index], cmap='gray')
windex = ipywidgets.IntSlider(index, min=0, max=ct.shape[0]-1, description='Frame', continuous_update=False)
woutput = ipywidgets.Output()
print(ct.shape)

(795, 405, 405)


In [12]:
@woutput.capture(clear_output=True, wait=True)
def func_ax(change):
    index = change.new
    im0.set_data(ct[index])
    display(fig)

# this chuck below will show you the CT data in sliced form along the z-axis. 

In [14]:
windex.observe(func_ax, names='value')
windex.notify_change({'name': 'value', 'new': windex.value, 'type': 'change'})
ipywidgets.VBox((windex, woutput))

# CT Cube for Testing, the anatomy of this cube is such that we want the RL model to learn how to center the anatomical structure in the FOV. The model needs to learn something that is robust and very unique, we think vertabrae? 


In [18]:
ct_cube = ct[191:484, :,:]


In [19]:
index_cube = ct_cube.shape[0] // 2
fig, axs = plt.subplots(ncols=1, figsize=(12,3), subplot_kw={'xticks':(),'yticks':()})
plt.subplots_adjust(wspace=0.1)
plt.close(fig)
# this shows the Sagittal slice. 
im0 = axs.imshow(ct_cube[index_cube], cmap='gray')
windex = ipywidgets.IntSlider(index_cube, min=0, max=ct_cube.shape[0]-1, description='Frame', continuous_update=False)
woutput = ipywidgets.Output()
print(ct_cube.shape)

(293, 405, 405)


In [20]:
@woutput.capture(clear_output=True, wait=True)
def func_ax(change):
    index = change.new
    im0.set_data(ct[index])
    display(fig)

In [21]:
windex.observe(func_ax, names='value')
windex.notify_change({'name': 'value', 'new': windex.value, 'type': 'change'})
ipywidgets.VBox((windex, woutput))

In [22]:

# Save the data as .png image. 
CT_save_path = r'C:\Users\asong18\Desktop\Deep_RL_View_Planning\CT_image_axial_vertabrae'

count = 0
for i in range (ct_cube.shape[0]):
    ct_slice = ct_cube[i]
    plt.imsave(CT_save_path + f"\cadaver_slice_{count}.png", ct_slice)
    count = count+1

# Find Rotation about the z-axis. 

Need to get this to work for the rotation experiment. 

Currently need the view to be what we are expecting. We are not getting proper resampling right now. 

In [23]:
ct_cube = ct[49:454, :,:]
theta_x = 0
theta_y = 45
theta_z = 0
width, height, depth = ct.shape

# create transform object.
euler_transform = itk.Euler3DTransform.New()


# set angle for rotation.
ct.TransformIndexToPhysicalPoint((int(np.ceil(width/2)), int(np.ceil(height/2)), int(np.ceil(depth/2))))
euler_transform.SetRotation(np.deg2rad(theta_x), np.deg2rad(theta_y), np.deg2rad(theta_z))
ct_cube = itk.resample_image_filter(ct_cube, output_parameters_from_image=ct_cube, transform=euler_transform)